# Build Model to Predict Discharge Based on SWE

##### Author: Kevin
##### Date: 10/17/2022
#### Objective: construct model to Predict Discharge with Current SWE features

In [1]:
# Import packages
import os
import h5py
import numpy as np
import pandas as pd
import copy
import sklearn

In [2]:
# Read Data
data = pd.read_csv('data/gage_with_swe1019.csv')
print('Total Number of rows:',len(data))

Total Number of rows: 86926


In [3]:
data.head()

,gage,time,ft,m3,ll_lon,ll_lat,tr_lon,tr_lat,swe_avg,swe_max
0,11402000,1984-10-01,54.0,1.529110,-121.157674,39.855478,-120.690823,40.049659,-1.0,-1
1,11402000,1984-10-02,52.0,1.472476,-121.157674,39.855478,-120.690823,40.049659,-1.0,-1
2,11402000,1984-10-03,49.0,1.387525,-121.157674,39.855478,-120.690823,40.049659,-1.0,-1
3,11402000,1984-10-04,49.0,1.387525,-121.157674,39.855478,-120.690823,40.049659,-1.0,-1
4,11402000,1984-10-05,48.0,1.359209,-121.157674,39.855478,-120.690823,40.049659,-1.0,-1


### Data Quality Check & Cleaning

In [4]:
# Replace -1 to missing values
data['swe_avg'] = data['swe_avg'].replace(-1,np.nan)

In [5]:
data = data.dropna(subset = ['swe_avg'])

In [6]:
data.iloc[96:101,:][['gage','time','m3','swe_avg','swe_max']]

,gage,time,m3,swe_avg,swe_max
188,11402000,1985-04-07,17.443178,236.664195,924
189,11402000,1985-04-08,16.338820,237.398308,925
190,11402000,1985-04-09,15.630899,246.025402,944
191,11402000,1985-04-10,15.800800,248.852392,951
192,11402000,1985-04-11,14.498225,249.257813,953


In [7]:
# Check Missing Value
data.isnull().sum()
### Result: 57 missing values in ft & 1 in m3

gage          0
time          0
ft         3617
m3         3561
ll_lon        0
ll_lat        0
tr_lon        0
tr_lat        0
swe_avg       0
swe_max       0
dtype: int64

In [8]:
# Show NA rows data of column m3
data[data['m3'].isna()]

,gage,time,ft,m3,ll_lon,ll_lat,tr_lon,tr_lat,swe_avg,swe_max
46750,11208000,2010-10-01,NaN,NaN,-118.818577,36.520114,-118.610906,36.677516,65.773254,1295
46751,11208000,2010-10-02,NaN,NaN,-118.818577,36.520114,-118.610906,36.677516,61.713703,1271
46752,11208000,2010-10-03,NaN,NaN,-118.818577,36.520114,-118.610906,36.677516,57.730436,1245
46753,11208000,2010-10-04,NaN,NaN,-118.818577,36.520114,-118.610906,36.677516,54.151617,1220
46754,11208000,2010-10-05,NaN,NaN,-118.818577,36.520114,-118.610906,36.677516,50.927660,1198
...,...,...,...,...,...,...,...,...,...,...
78886,11189500,1996-09-26,NaN,NaN,-118.383732,35.728555,-118.003533,36.437843,0.406805,58
78887,11189500,1996-09-27,NaN,NaN,-118.383732,35.728555,-118.003533,36.437843,0.393276,57
78888,11189500,1996-09-28,NaN,NaN,-118.383732,35.728555,-118.003533,36.437843,0.367547,55
78889,11189500,1996-09-29,NaN,NaN,-118.383732,35.728555,-118.003533,36.437843,0.334695,52


In [9]:
def filter_dates(data,gage,date,s_or_g):
    '''
        Objective: Filter out data that has missing values in Water Discharge
        Input:
            - data: gage_with_swe.csv {pandas dataframe}
            - gage: gage id (ex: 11208000) {int}
            - date: date (ex: 'YYYY-mm-dd') {str}
            - s_or_g: 's'(smaller) or 'g'(greater) Please only input s or g.
    '''
    # 1. Delete all rows with gage xx
    new = data.drop(data.loc[data['gage']==gage].index)
    # 2. Get Interested Dates
    if s_or_g == 's':
        add = data[(data['gage'] == gage) & (pd.to_datetime(data['time']) <= pd.to_datetime(date))]
    elif s_or_g == 'g':
        add = data[(data['gage'] == gage) & (pd.to_datetime(data['time']) >= pd.to_datetime(date))]
    else:
        print("ERROR in Code. Please specify either s or g for the 4th parameter")
    # 3. add data back to new
    new = pd.concat([new,add],axis=0)
    
    return new
    

In [10]:
# Filter out dates specified by Zixi 
data = filter_dates(data,11208000,'2002-07-01','s')
data = filter_dates(data,11202710,'1992-01-01','g')
data = filter_dates(data,11185500,'2013-01-01','s')
data = filter_dates(data,11189500,'1997-01-01','g')

In [11]:
# Check Missing Value
data.isnull().sum()

gage       0
time       0
ft         0
m3         1
ll_lon     0
ll_lat     0
tr_lon     0
tr_lat     0
swe_avg    0
swe_max    0
dtype: int64

In [12]:
data[data['m3'].isna()]

,gage,time,ft,m3,ll_lon,ll_lat,tr_lon,tr_lat,swe_avg,swe_max
52593,11202710,1992-09-30,13.0,NaN,-118.72619,36.085359,-118.5279,36.325132,1.681694,101


In [13]:
# fill the missing value
data.loc[52593,'m3'] = 0.368119

In [14]:
# Drop ft column & Reset Index
data = data.drop(['ft'],axis=1)
data = data.reset_index(drop=True)

In [15]:
# Take only interested columns
data_new = data[['time','gage','m3','swe_avg','swe_max']]

In [16]:
# # Plot correlation of swe_avg lag to m3
# data_new = data_new.drop(['swe_max'],axis=1)
# avg_cor_values = data_new.corr().iloc[0,1:]

In [17]:
# # Correlation between swe_avg lag & water discharge
# avg_cor_values

--------------------------------------------
### Focus on Gage 11402000
Create ML algorithm based model

- Input: SWE avg & max, swe lag features
- Output: gage dicharage rate

In [18]:
# gage_data: focus on 11402000
gage_data = data_new[data_new['gage'] == 11402000]

#### 1. Add Lag Features

In [19]:
import warnings
warnings.filterwarnings("ignore")
### 1. Feature Engineering:  Lag Features with 14 days
lag_days = [1,2,3,4,5,6,7,15,30,40,60,75,90]
for i in lag_days:
    gage_data['swe_avg_lag{}'.format(i)] = gage_data['swe_avg'].shift(i)

In [20]:
gage_data.head()

,time,gage,m3,swe_avg,swe_max,swe_avg_lag1,swe_avg_lag2,swe_avg_lag3,swe_avg_lag4,swe_avg_lag5,swe_avg_lag6,swe_avg_lag7,swe_avg_lag15,swe_avg_lag30,swe_avg_lag40,swe_avg_lag60,swe_avg_lag75,swe_avg_lag90
0,1985-01-01,11402000,2.690100,0.000741,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985-01-02,11402000,2.605150,0.000000,0,0.000741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985-01-03,11402000,2.605150,0.000000,0,0.000000,0.000741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1985-01-04,11402000,2.605150,0.000000,0,0.000000,0.000000,0.000741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1985-01-05,11402000,2.548516,0.000000,0,0.000000,0.000000,0.000000,0.000741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 2. Shift Target Variable by One day

In [21]:
## Need to predict next day, to shift target variable one day
gage_data['m3_target'] = gage_data['m3'].shift(1)

In [22]:
# Delete NA values
gage_data = gage_data.dropna()
# Reset Index after dropping
gage_data = gage_data.reset_index(drop=True)
gage_data.head()

,time,gage,m3,swe_avg,swe_max,swe_avg_lag1,swe_avg_lag2,swe_avg_lag3,swe_avg_lag4,swe_avg_lag5,swe_avg_lag6,swe_avg_lag7,swe_avg_lag15,swe_avg_lag30,swe_avg_lag40,swe_avg_lag60,swe_avg_lag75,swe_avg_lag90,m3_target
0,1985-04-01,11402000,12.289511,237.177432,924,237.091091,237.256742,237.448152,237.599275,237.752540,237.712473,237.810760,204.030071,153.854973,44.145301,0.757538,4.272533,0.000741,9.259609
1,1985-04-02,11402000,15.432681,237.075438,924,237.177432,237.091091,237.256742,237.448152,237.599275,237.752540,237.712473,234.348465,154.983440,51.167353,0.719723,12.847751,0.000000,12.289511
2,1985-04-03,11402000,17.839613,236.966002,924,237.075438,237.177432,237.091091,237.256742,237.448152,237.599275,237.752540,238.530345,156.385236,52.312490,2.530098,12.393063,0.000000,15.432681
3,1985-04-04,11402000,17.414861,236.945570,924,236.966002,237.075438,237.177432,237.091091,237.256742,237.448152,237.599275,238.493739,159.831274,54.186824,2.858406,12.664991,0.000000,17.839613
4,1985-04-05,11402000,17.188326,236.737189,924,236.945570,236.966002,237.075438,237.177432,237.091091,237.256742,237.448152,238.450953,159.694129,77.601087,2.751909,10.803702,0.000000,17.414861


#### 3. Add Differencing

In [ ]:
# differencing
interval = 5
diff_list = []
for i in range(interval,len(gage_data)):
    diff_list.append(gage_data.loc[i,'m3'] - gage_data.loc[i-interval,'m3'])

In [ ]:
### Assign a new variable named "diff"
gage_data['m3_diff'] = 0
gage_data.loc[interval:len(gage_data),'m3_diff'] = pd.Series(diff_list[:len(diff_list)-interval])
gage_data = gage_data.iloc[interval-1:,]

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(10,5))
plt.plot(gage_data['m3_diff'])
plt.gcf().autofmt_xdate()
plt.xlabel("Year")
plt.ylabel("Differencing")
plt.show()

#### 4. Conduct Mickey Fuller Test on Target Variable

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(gage_data['m3'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

In [ ]:
gage_data['time'] = pd.to_datetime(gage_data['time'])
gage_data = gage_data.set_index(gage_data['time'])

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(figsize=(10,5))
plt.plot(gage_data['m3'])
plt.gcf().autofmt_xdate()
plt.xlabel("Year")
plt.ylabel("Water Discharge")
plt.title("Water Discharge Over Time for Gage 11402000")
plt.show()

**NOTE**: p-value <= 0.05: the data does not have a unit root and is stationary

#### 5. Auto-Correlation

In [ ]:
## Auto Correlation for 100 days
# Auto Correlation: measures the relationship between a variable’s current values and its historical values.
import statsmodels.api as sm
from statsmodels.graphics import tsaplots
ac = sm.tsa.acf(gage_data['m3_target'],nlags = 50)
tsaplots.plot_acf(gage_data['m3_target'], lags=100,title = 'Auto-Correlation on Water Discharge (11402000)')

### Data Splitting

In [23]:
# Order Data by Date
gage_data['time'] = pd.to_datetime(gage_data['time'])
gage_data = gage_data.set_index(gage_data['time'])
gage_data = gage_data.sort_index()

In [24]:
gage_data.head()

,time,gage,m3,swe_avg,swe_max,swe_avg_lag1,swe_avg_lag2,swe_avg_lag3,swe_avg_lag4,swe_avg_lag5,swe_avg_lag6,swe_avg_lag7,swe_avg_lag15,swe_avg_lag30,swe_avg_lag40,swe_avg_lag60,swe_avg_lag75,swe_avg_lag90,m3_target
time,,,,,,,,,,,,,,,,,,,
1985-04-01,1985-04-01,11402000,12.289511,237.177432,924,237.091091,237.256742,237.448152,237.599275,237.752540,237.712473,237.810760,204.030071,153.854973,44.145301,0.757538,4.272533,0.000741,9.259609
1985-04-02,1985-04-02,11402000,15.432681,237.075438,924,237.177432,237.091091,237.256742,237.448152,237.599275,237.752540,237.712473,234.348465,154.983440,51.167353,0.719723,12.847751,0.000000,12.289511
1985-04-03,1985-04-03,11402000,17.839613,236.966002,924,237.075438,237.177432,237.091091,237.256742,237.448152,237.599275,237.752540,238.530345,156.385236,52.312490,2.530098,12.393063,0.000000,15.432681
1985-04-04,1985-04-04,11402000,17.414861,236.945570,924,236.966002,237.075438,237.177432,237.091091,237.256742,237.448152,237.599275,238.493739,159.831274,54.186824,2.858406,12.664991,0.000000,17.839613
1985-04-05,1985-04-05,11402000,17.188326,236.737189,924,236.945570,236.966002,237.075438,237.177432,237.091091,237.256742,237.448152,238.450953,159.694129,77.601087,2.751909,10.803702,0.000000,17.414861


In [25]:
train_size = int(len(gage_data) *0.8)

In [26]:
train = gage_data.iloc[:train_size,:]
test  = gage_data.iloc[train_size:,:]
print(f'training size: {len(train)} ({round(len(train)/len(gage_data),2)})')
print(f'test size: {len(test)} ({round(len(test)/len(gage_data),2)})')

training size: 9278 (0.8)
test size: 2320 (0.2)


In [27]:
# Shuffle Data
train = train.sample(frac=1)
test = test.sample(frac=1)

In [29]:
train[['gage','m3_target','swe_avg','swe_avg_lag1','swe_avg_lag2','swe_avg_lag3','swe_avg_lag4','swe_avg_lag5']]

,gage,m3_target,swe_avg,swe_avg_lag1,swe_avg_lag2,swe_avg_lag3,swe_avg_lag4,swe_avg_lag5
time,,,,,,,,
1994-08-27,11402000,0.235030,31.557945,33.839457,36.861015,40.357363,43.718185,46.632669
2008-10-27,11402000,0.829684,0.869171,0.925221,0.986461,1.055116,1.128714,1.203466
2000-01-18,11402000,10.194065,0.155764,0.166063,0.176333,0.187126,0.203356,0.226808
1993-09-10,11402000,0.821189,300.898391,312.522327,322.833086,331.695309,337.590020,340.215522
1994-02-22,11402000,5.323567,0.904789,0.418026,0.426924,0.435904,0.448591,0.472593
...,...,...,...,...,...,...,...,...
1987-03-09,11402000,9.231292,1.428132,1.590981,1.740457,0.157742,0.084555,0.110617
2006-07-18,11402000,1.565922,619.341105,591.400972,540.762921,542.454688,552.869281,559.704097
1991-09-01,11402000,0.396436,44.344346,47.762701,49.918740,49.766463,52.528588,55.836711


In [30]:
train.columns

Index(['time', 'gage', 'm3', 'swe_avg', 'swe_max', 'swe_avg_lag1',
       'swe_avg_lag2', 'swe_avg_lag3', 'swe_avg_lag4', 'swe_avg_lag5',
       'swe_avg_lag6', 'swe_avg_lag7', 'swe_avg_lag15', 'swe_avg_lag30',
       'swe_avg_lag40', 'swe_avg_lag60', 'swe_avg_lag75', 'swe_avg_lag90',
       'm3_target'],
      dtype='object')

In [36]:
# Get X & Y
feature_cols = ['swe_avg', 'swe_max', 'swe_avg_lag1',
       'swe_avg_lag2', 'swe_avg_lag3', 'swe_avg_lag4', 'swe_avg_lag5',
       'swe_avg_lag6', 'swe_avg_lag7', 'swe_avg_lag15', 'swe_avg_lag30',
       'swe_avg_lag40', 'swe_avg_lag60', 'swe_avg_lag75', 'swe_avg_lag90']
train_x = train[feature_cols]
train_y = train['m3_target']
test_x = test[feature_cols]
test_y = test['m3_target']

### Model Building

#### Baseline (Average)

In [37]:
print(train_y.mean())
print(test_y.mean())

6.749982891979906
5.161516071096004


In [38]:
### Function for Model Evaluation
from sklearn.metrics import mean_squared_error
import math

def mape(actual, pred): 
    ## Calculating Mean Absolute Percentage Error
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

def relative_root_mean_squared_error(true, pred):
    num = np.sum(np.square(true - pred))
    den = np.sum(np.square(pred))
    squared_error = num/den
    rrmse_loss = np.sqrt(squared_error)
    return rrmse_loss

def evaluation(model,train_x,train_y,test_x,_test_y):
    train_pred = model.predict(train_x)
    train_pred = np.where(train_pred<0,0,train_pred)
    test_pred = model.predict(test_x)
    test_pred = np.where(test_pred<0,0,test_pred)
    mse_train = mean_squared_error(train_y,train_pred)
    mse_test = mean_squared_error(test_y,test_pred)
    rrmse_train = relative_root_mean_squared_error(train_y, train_pred)
    rrmse_test = relative_root_mean_squared_error(test_y,test_pred)
    

    print('Root Mean Squared Error on Train:',math.sqrt(mse_train))
    print('MApe on Train:',mape(train_y,train_pred))
    print('RRMSE on Train:',rrmse_train)
    print("Root Mean Squared Error on Test:",math.sqrt(mse_test))
    print('MApe on Test:',mape(test_y,test_pred))
    print('RRMSE on Test:',rrmse_test)
    
    
    return test_pred

#### 1. Linear Regression

In [39]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
# Fit model
lr.fit(train_x,train_y)
# Evaluation
lr_pred_test = evaluation(lr,train_x,train_y,test_x,test_y)

Root Mean Squared Error on Train: 16.176164169094253
MApe on Train: 273.2483744501333
RRMSE on Train: 2.0559041946678738
Root Mean Squared Error on Test: 9.910594460939247
MApe on Test: 532.791630683554
RRMSE on Test: 1.3072871473787053


In [40]:
# check test data result
test_result = pd.concat([test_y.reset_index(drop=True),pd.DataFrame(lr_pred_test)],axis=1)
test_result.columns = ['True', "Prediction"]
test_result

,True,Prediction
0,11.921392,6.875487
1,7.334063,7.996832
2,3.567923,7.310674
3,10.307332,7.585361
4,2.231368,7.848113
...,...,...
2315,1.648040,5.258422
2316,0.470060,4.615857
2317,5.889904,11.207942
2318,1.758476,0.000000


##### Analysis

In [ ]:
### Correlation
cols_interested = ['m3_target','swe_avg','swe_max','swe_avg_lag1','swe_avg_lag2','swe_avg_lag3','swe_avg_lag4','swe_avg_lag5',
           'swe_avg_lag6','swe_avg_lag7','swe_avg_lag8','swe_avg_lag9','swe_avg_lag10','swe_avg_lag11',
           'swe_avg_lag12','swe_avg_lag13','swe_avg_lag14']
df = train[cols_interested]
vi = df.corr().iloc[0,1:]
vi= pd.DataFrame(vi)
vi.columns=['Feature_Importance']
vi

In [ ]:
train_x.columns

In [ ]:
# regression coefficients
lr_coef = pd.DataFrame(lr.coef_)
lr_coef.index = train_x.columns
lr_coef.columns = ['Linear_Regression_Coefficients']

In [ ]:
pd.concat([vi,lr_coef],axis=1)

In [ ]:
# Residual = observed - predict
residual = np.array(test_y) - lr_pred_test

In [ ]:
### Residual Plot

plt.scatter(lr_pred_test,residual)
plt.xlabel('y-predict')
plt.ylabel('Residual')
plt.title('Residual Plot Between Predicted and Actual Value')
plt.show()

#### 2. Random Forest


In [41]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
# Fit model
rf.fit(train_x,train_y)

RandomForestRegressor()

In [42]:
# Evaluation
rf_pred_test = evaluation(rf,train_x,train_y,test_x,test_y)

Root Mean Squared Error on Train: 3.627033462861453
MApe on Train: 15.53734741364374
RRMSE on Train: 0.2295428518388673
Root Mean Squared Error on Test: 11.672537347634174
MApe on Test: 464.751228892072
RRMSE on Test: 1.0426739146110373


#### 3. XGBoost

In [ ]:
####

1. Do the differencing
2. Statistical Test (Dickey–Fuller test) to 
3. Add lag features of days (30,60,90,120)